In [15]:
import os
import vertexai
from dotenv import load_dotenv
from IPython.display import Markdown, Video, display
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)
import google.generativeai as genai
     

load_dotenv(override=True)

c:\Users\User\gemini-examples\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Set Google cloud project information and initialize vertexAI SDK

To get started using Vertex AI , you must have an existing Google Cloud project and enable the vertexAI API.

In [ ]:
# Define project information
PROJECT_ID = os.getenv("PROJECT_ID")  
LOCATION = os.getenv("LOCATION")
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [25]:
model = GenerativeModel("gemini-1.5-pro")

generation_config = GenerationConfig(temperature=1, top_p=0.95, max_output_tokens=8192)

safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

In [26]:
def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for display.
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

In [27]:
video_analysis_prompt = """You are an expert in politics and history. Provide a detailed analysis of the video including each speakers facial expressions and mood at key points."""
     

# Load file directly from Google Cloud Storage
video_uri = "https://storage.googleapis.com/ds-genai-test-renault-ad-lib-video/a41201dc-69bc-4c74-b43b-ff54c6759bc8.mp4"

# Load contents
contents = [
    Part.from_uri(
        uri=video_uri,
        mime_type="video/mp4",
    ),
    video_analysis_prompt,
]

# Display the Video
display(Video(get_url_from_gcs(video_uri)))

In [28]:
# Send to Gemini
response = model.generate_content(contents, generation_config=generation_config)

# Display results
display(Markdown(response.text))

The video features a single speaker, Cléa Martinet, VP Sustainability, Renault Group & Ampere. The video appears to be promotional for Renault's sustainability initiatives.

**Initial Mood:**

Cléa starts with a playful, almost surprised expression as she reads a message on her phone prompting her to talk about "Planet." She then smiles warmly and confidently says, "Okay, right on the topic. Let's go." This shift indicates her enthusiasm and readiness to engage with the topic of sustainability.

**Throughout the Interview:**

* **Enthusiastic & Passionate:** Cléa consistently maintains a positive and enthusiastic demeanor. Her smiles are frequent and genuine, suggesting a genuine passion for her work. When she describes herself as "enthusiastic," it’s reflected in her bright expression.
* **Confident & Knowledgeable:** She speaks clearly and confidently, using hand gestures to emphasize her points. This indicates her knowledge and expertise in the field of sustainability. She exhibits a calm confidence while explaining the "three paradoxes" of electric vehicle production, indicating her mastery of the subject.
* **Thoughtful & Reflective:**  When discussing the challenges of EV production, her expression becomes more serious and she uses more deliberate hand gestures. This suggests that she is giving careful consideration to the complexities involved. 
* **Positive & Optimistic:** Despite the challenges, she ends on a positive note, expressing her happiness with Renault's approach to sustainability and hope for the future. When she talks about the R5 and its history, a hint of nostalgia appears in her eyes alongside a contented smile.

**Overall Impression:**

Cléa Martinet comes across as a highly credible and engaging speaker. Her passion for sustainability, combined with her expertise and optimism, makes her an effective advocate for Renault's initiatives.  Her facial expressions and tone of voice throughout the video effectively convey her personality and enhance her message.

**Absence of Second Speaker:**

Although the initial part of the video shows Cléa receiving a message from "Aurélien," there is no second speaker present.  The message acts as a prompt to initiate the interview and set the topic.  The brief interaction adds a touch of spontaneity and  introduces the video's format.  It also avoids the formality of a traditional interview setting, creating a more relatable and informal tone. 


In [29]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "The video features a single speaker, Cléa Martinet, VP Sustainability, Renault Group & Ampere. The video appears to be promotional for Renault\'s sustainability initiatives.\n\n**Initial Mood:**\n\nCléa starts with a playful, almost surprised expression as she reads a message on her phone prompting her to talk about \"Planet.\" She then smiles warmly and confidently says, \"Okay, right on the topic. Let\'s go.\" This shift indicates her enthusiasm and readiness to engage with the topic of sustainability.\n\n**Throughout the Interview:**\n\n* **Enthusiastic & Passionate:** Cléa consistently maintains a positive and enthusiastic demeanor. Her smiles are frequent and genuine, suggesting a genuine passion for her work. When she describes herself as \"enthusiastic,\" it’s reflected in her bright expression.\n* **Confident & Knowledgeable:** She speaks clearly and confidently, using hand gestures to emphasize her points. Thi

In [ ]:
# TODO mettre en place un pricing

In [34]:
print(f"Prompt Token Count: {model.count_tokens(contents).total_tokens}")
print(f"Prompt Character Count: {model.count_tokens(contents).total_billable_characters}")

print(f"Prompt Token Count: {response.usage_metadata.prompt_token_count}")
print(f"Candidates Token Count: {response.usage_metadata.candidates_token_count}")
print(f"Total Token Count: {response.usage_metadata.total_token_count}")

price = model.count_tokens(contents).total_tokens * 1.25 /1000000
print("Price: ", price)

Prompt Token Count: 53127
Prompt Character Count: 126
Prompt Token Count: 53127
Candidates Token Count: 478
Total Token Count: 53605
Price:  0.06640875
